In [12]:
import pandas as pd
import wave
import numpy as np
import matplotlib.pyplot as plt


# 2. Find sampling rate + number samples + number channels and save information in df

In [13]:
other = pd.read_csv('data_frames/ecoGuns_SST_Guns_Training-OtherSounds.csv')
ecoguns = pd.read_csv('data_frames/Guns_Training_ecoGuns_SST.csv')
pnn = pd.read_csv('data_frames/nn_Grid50_guns_dep1-7_Guns_Training.csv')


In [14]:
def sampling_rate_and_number_samples_channels (instance_of_df):
    clip_location = instance_of_df['Clip Location']
    temp = wave.open(clip_location, 'rb') # rb : read and write to that file
    sample_rate = temp.getframerate() # returns sampling frequency = how many samples per second
    number_samples = temp.getnframes() # total number of frames in this recording = total number of samples
    channel = temp.getnchannels()
    return sample_rate, number_samples, channel

def sampling_all (df):
    df['Sampling Rate/Frequency'], df['Total number of Samples'], df['Channel'] = zip(*df.apply(sampling_rate_and_number_samples_channels, axis = 1))


In [15]:
sampling_all(other)
sampling_all(ecoguns)
sampling_all(pnn)

# update Dfs
other.to_csv('data_frames/ecoGuns_SST_Guns_Training-OtherSounds.csv')
ecoguns.to_csv('data_frames/Guns_Training_ecoGuns_SST.csv')
pnn.to_csv('data_frames/nn_Grid50_guns_dep1-7_Guns_Training.csv')


# 3. Plot time + freq domain


In [18]:
# https://learnpython.com/blog/plot-waveform-in-python/

def plot_time (origen_file, destination_file):
    wave_object = wave.open(origen_file, 'rb')
    channel = wave_object.getnchannels()
    n_samples = wave_object.getnframes()
    sample_freq = wave_object.getframerate()
    t_audio = n_samples/sample_freq
    signal_wave = wave_object.readframes(n_samples)
    signal_array = np.frombuffer(signal_wave, dtype=np.int16)

    if channel !=1:
        raise Exception ('Some sample have more than one channel')
    times = np.linspace(0, t_audio, num = n_samples)
    plt.figure(figsize=(15, 5))
    plt.plot(times, signal_array)
    plt.title('Audio Wave')
    plt.ylabel('Signal Value')
    plt.xlabel('Time (s)')
    plt.xlim(0, t_audio)
    plt.grid()
    #plt.savefig(destination_file)
    #plt.show()

def plot_freq_spectrum (origen_file, destination_file):
    wave_object = wave.open(origen_file, 'rb')
    channel = wave_object.getnchannels()
    n_samples = wave_object.getnframes()
    sample_freq = wave_object.getframerate()
    t_audio = n_samples/sample_freq
    signal_wave = wave_object.readframes(n_samples)
    if channel !=1:
        raise Exception ('Some sample have more than one channel')
    signal_array = np.frombuffer(signal_wave, dtype=np.int16)
    plt.figure(figsize=(19, 5))
    plt.specgram(signal_array, Fs=sample_freq, vmin=-20, vmax=50)
    #plt.specgram(signal_array, Fs=sample_freq, vmin=0, vmax=2)

    plt.title('Spectogram')
    plt.ylabel('Frequency (Hz)')
    plt.xlabel('Time (s)')
    plt.xlim(0, t_audio)
    plt.colorbar()
    #plt.savefig(destination_file)
    #plt.show()



In [ ]:

for i in range (20):
    # Pnn
    file_name = pnn.iloc[i]['Clip Location']
    plot_freq_spectrum(file_name, 'data/Images/gunshot/freq_' + file_name.replace('.wav', '.png').replace('data/Clips/', ''))
    plot_time(file_name, 'data/Images/gunshot/time_' + file_name.replace('.wav', '.png').replace('data/Clips/', '' ))
    # Ecoguns
    file_name = ecoguns.iloc[i]['Clip Location']
    plot_freq_spectrum(file_name, 'data/Images/gunshot/freq_' + file_name.replace('.wav', '.png').replace('data/Clips/', ''))
    plot_time(file_name, 'data/Images/gunshot/time_' + file_name.replace('.wav', '.png').replace('data/Clips/', '' ))
    # Others
    file_name = other.iloc[i]['Clip Location']
    plot_freq_spectrum(file_name, 'data/Images/no_gunshot/freq_' + file_name.replace('.wav', '.png').replace('data/Clips/', ''))
    plot_time(file_name, 'data/Images/no_gunshot/time_' + file_name.replace('.wav', '.png').replace('data/Clips/', '' ))
